In [1]:
from gurobipy import Model, GRB
import pandas as pd
from datetime import datetime

In [2]:
# Définir un modèle Gurobi
model = Model("planification_taches_machines")

Set parameter Username
Set parameter LicenseID to value 2617727
Academic license - for non-commercial use only - expires 2026-02-03


In [3]:
# Charger les données
chantiers_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/fret_sncf/instance_WPY_realiste_jalon1.xlsx', sheet_name='Chantiers')
machines_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/fret_sncf/instance_WPY_realiste_jalon1.xlsx', sheet_name='Machines')
sillons_arrivee_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/fret_sncf/instance_WPY_realiste_jalon1.xlsx', sheet_name='Sillons arrivee')
sillons_depart_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/fret_sncf/instance_WPY_realiste_jalon1.xlsx', sheet_name='Sillons depart')
correspondances_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/fret_sncf/instance_WPY_realiste_jalon1.xlsx', sheet_name='Correspondances')
taches_humaines_df = pd.read_excel('/Users/thomasdesforges/Desktop/CS/2A/ST7/Projet ST7/fret_sncf/instance_WPY_realiste_jalon1.xlsx', sheet_name='Taches humaines')

# Variables de décision : x_m,h,t = 1 si la machine m travaille sur le wagon w à l'heure h, 0 sinon
# Listes des machines, heures et wagons
machines = machines_df['Machine'].tolist()
trains = sillons_arrivee_df['n°TRAIN'].tolist()
hours = range(24 * 60)  # De 0 à 1440 minutes (pour chaque minute de la journée)



In [4]:
# Créer des variables de décision pour chaque machine, heure et wagon
x = {}
for m in machines:
    for h in hours:
        for t in trains:
            x[(m, h, t)] = model.addVar(vtype=GRB.BINARY, name=f"x_{m}_{h}_{t}")

# Affichage des variables créées (facultatif pour vérification)
print(f"Variables de décision créées : {len(x)}")

Variables de décision créées : 164160


In [5]:
# Fonction pour convertir une heure au format "HH:MM" en minutes depuis minuit
def time_to_minutes(time_str):
    time_part = time_str.split(',')[1] if ',' in time_str else time_str
    time_part = time_part.strip('()')  # Enlever les parenthèses
    time_obj = datetime.strptime(time_part, "%H:%M")
    return time_obj.hour * 60 + time_obj.minute

In [6]:
# Contrainte 1: Chaque tâche doit être attribuée à une machine et à un wagon à un créneau horaire spécifique
for _, row in machines_df.iterrows():
    task_type = row['Type de tache']
    unavailable_times = row['Indisponibilites']
    
    if unavailable_times != 0:
        periods = unavailable_times.strip('()').split(';')
        for period in periods:
            day_of_week, time_str = period.split(',')
            start_time = time_to_minutes(time_str.split('-')[0])
            end_time = time_to_minutes(time_str.split('-')[1])
            
            # Contrainte de disponibilité de la machine pendant les périodes d'indisponibilité
            for h in range(start_time, end_time):
                for t in trains:
                    model.addConstr(sum(x[(m, h, t)] for m in machines if m == task_type) == 0)



* Contrainte 2: Chaque wagon doit avoir une machine affectée à un moment donné
$$
    \quad \forall h, \quad \forall t, \sum_{m} x_{m,h,t} \leq 1, 
$$


In [7]:
# Contrainte 2: Chaque wagon doit avoir une machine affectée à un moment donné
for h in hours:
    for t in trains:
        # La machine ne peut travailler que sur un wagon à un moment donné
        model.addConstr(sum(x[(m, h, t)] for m in machines) <= 1)

* Contrainte 3: Capacité des chantiers (respecter le nombre de voies disponibles)
$$
\quad \forall m, \quad \forall t, \sum_{m} \sum_{t} x_{m, h, t} \leq nb_{voies disponibles}
$$

In [15]:
# Contrainte 3: Capacité des chantiers (respecter le nombre de voies disponibles)
# Modifier cette contrainte pour que le wagon soit attribué à un chantier spécifique
for _, row in chantiers_df.iterrows():
    chantier = row['Chantier']
    voies_disponibles = row['Nombre de voies']
    
    # Filtrer les wagons associés à ce chantier
    wagons_in_chantier = [t for t in trains if t == chantier]  # Exemple de correspondance, à ajuster selon vos données

    # Calculer le nombre de wagons occupés sur chaque chantier à un moment donné
    for h in hours:
        model.addConstr(sum(x[(m, h, t)] for m in machines for t in wagons_in_chantier) <= voies_disponibles)

* Contrainte 4: Délai de 60 minutes entre arrivée et débranchement
$$
\quad \forall h \in \left[0,59 \right]  x_{DEB, arrival\_time + h, t} = 0
$$

In [20]:
# Contrainte 4: Parallélisation des tâches humaines et machines (délai de 60 minutes entre arrivée et débranchement)
for _, row in sillons_arrivee_df.iterrows():
    train = row['n°TRAIN']
    arrival_time = time_to_minutes(row['HARR'])    
    # Permettre à n'importe quelle machine de travailler sur le wagon, pas seulement "DEB"
    for h in range (0, 60):
        model.addConstr(x[('DEB', (arrival_time + h)%1439, train)]  == 0)
                  


* Contrainte 5: Délai de 150 minutes entre formation et dégarage
$$
\quad \forall h \in \left[1,150 \right]  x_{FOR, time\_DEG - h, t} = 0
$$

In [10]:
# Contrainte 5: Parallélisation des tâches humaines et machines (délai de 150 minutes entre formation et dégarage)
for _, row in sillons_departure_df.iterrows():
    train = row['n°TRAIN']
    departure_time = time_to_minutes(row['HDEP'])
    # Permettre à n'importe quelle machine de travailler sur le wagon, pas seulement "DEB"
    for h in range (1,150):
        model.addConstr(x[('FOR', time_DEG_start - h, train)]  == 0)
          


* Contrainte 6: Délai de 20 minutes entre dégarage et départ
$$
\quad \forall h \in \left[1,20 \right]  x_{DEG, departure\_time - h, t} = 0
$$

In [14]:
# Contrainte 6: Parallélisation des tâches humaines et machines (délai de 20 minutes entre dégarage et départ)
for _, row in sillons_depart_df.iterrows():
    train = row['n°TRAIN']
    departure_time = time_to_minutes(row['HDEP'])
    # Permettre à n'importe quelle machine de travailler sur le wagon, pas seulement "DEB"
    for h in range(1,20):
        model.addConstr(x[('DEG', departure_time - h, train)]  == 0)

TypeError: 'bool' object is not iterable

In [ ]:
# Optimisation
model.setObjective(0, GRB.MINIMIZE)  # Pas d'optimisation spécifique ici pour ce jalon

# Résolution du problème
model.optimize()

# Afficher les résultats
if model.status == GRB.OPTIMAL:
    for (m, h, t), var in x.items():
        if var.x > 0.5:  # Si la machine travaille sur le wagon à cette heure
            print(f"Machine {m} travaille sur le train {t} à la minute {h}")
else:
    print("Aucune solution optimale trouvée.")

In [ ]:
#model.addConstr(row['HDEP']-row['HARR']>=(60+15+15+150+15+20)) #diff_dep_arr>=(60+15+15+150+15+20))